In [1]:
from model import baseline_resnet18

model = baseline_resnet18(pretrained=True).cuda()

In [45]:
from coco_dataset import CocoPoseDataset, convert_coco_annotations

In [2]:
import torch
import torchvision.transforms
from coco_dataset import CocoPoseDataset, convert_coco_annotations

INPUT_SHAPE = (256, 256)

transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(INPUT_SHAPE),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
    
dataset = CocoPoseDataset(
    root='train2017',
    annotations='annotations/person_keypoints_train2017.json',
    shape=(INPUT_SHAPE[0] // 4, INPUT_SHAPE[1] // 4),
    stdev=1.5,
    transforms=transforms
)

loading annotations into memory...
Done (t=5.66s)
creating index...
index created!


In [40]:
loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=16,
    shuffle=True,
    num_workers=0
)

In [46]:
image, target = dataset.coco_detection[0]

In [54]:
import time

t0 = time.time()
for i in range(50):
    convert_coco_annotations((image.height, image.width), target, dataset.topology)
t1 = time.time()

print((t1 - t0) / 50.0)

0.002111668586730957
